In [1]:
import pandas as pd
import numpy as np

In [2]:
def clean_wiki(df):
    for c in df.columns:
        try:
            df[c] = df[c].str.replace('http://www.wikidata.org/entity/','')
        except:
            pass
    return df

In [25]:
raw_df = pd.read_csv('../data/entities/Q30.csv')
#before all else
raw_df = clean_wiki(raw_df)
[_ for _ in raw_df.columns if 'value' in _]

['business.value',
 'businessLabel.value',
 'country.value',
 'industries.value',
 'ceos.value',
 'chairs.value',
 'hqs.value',
 'groups.value',
 'origindate.value',
 'officialname.value',
 'employees.value',
 'profit.value',
 'markcap.value',
 'assets.value',
 'equity.value']

In [26]:
new_names = {'business.value':'id',
'businessLabel.value':'name',
'origindate.value':'start_date',
'country.value':'country_vals',
'industries.value':'industry_vals',
'ceos.value':'ceo_vals',
'chairs.value':'chair_vals',
'hqs.value':'hq_vals',
'groups.value':'group_vals',
'employees.value':'employee_count_quant',
'profit.value':'profit_quant',
'assets.value':'assets_quant', 
'equity.value':'equity_quant',
'markcap.value':'market_cap_quant'}

In [27]:
df = pd.DataFrame(columns=list(new_names))
prev_cols = [_ for _ in list(new_names) if _ in raw_df.columns]
df[prev_cols] = raw_df[prev_cols]
df.columns = df.columns.map(new_names)
val_cols = [_ for _ in df.columns if '_vals' in _]
quant_cols = [_ for _ in df.columns if '_quant' in _]
for col in quant_cols:
    labels = [str(_)+'_'+col.replace('_quant','') for _ in range(0,10)]
    df[col] = pd.cut(df[col].fillna(0), 10, labels=labels)
val_cols += quant_cols
ent_df = df.drop(val_cols,axis=1)
ent_df = ent_df.drop_duplicates().fillna('')
ent_df

,id,name,start_date
0,Q54833437,United Way of East Central Iowa,
1,Q54556904,United Way of Berks County,
2,Q55665807,Kiowa County Press,
3,Q63342307,Warrior Lacrosse,
4,Q1583249,USC School of Cinematic Arts,1929-01-01T00:00:00Z
...,...,...,...
19158,Q54670069,Catholic Charities of the Diocese of Santa Rosa,
19159,Q7199292,Pittsburgh Glass Center,2001-01-01T00:00:00Z
19160,Q55668066,Exeter News-Letter,
19161,Q71042495,V.I.M.,1977-01-01T00:00:00Z


In [28]:
mult_df = df[val_cols+['id']]
redex = mult_df.copy()
redex = redex.replace(np.nan, '')
redex.index = mult_df['id']

In [53]:
def dic_to_ls(dic,typ,node_typ):
    ret_ls = []
    for k, vs in dic.items():
        for v in vs:
            ret_ls.append({'company_id':k,'val_id':v,'edge_typ':typ,'node_typ':node_typ})
    return ret_ls

In [56]:
ls = dic_to_ls(redex['industry_vals'].dropna().str.split(',').to_dict(), 'INDUSTRY', 'Industry') \
+ dic_to_ls(redex['ceo_vals'].dropna().str.split(',').to_dict(), 'CEO', 'Person')\
+ dic_to_ls(redex['chair_vals'].dropna().str.split(',').to_dict(), 'CHAIRMAN', 'Person')\
+ dic_to_ls(redex['hq_vals'].dropna().str.split(',').to_dict(), 'HEADQUARTERS', 'Location')\
+ dic_to_ls(redex['group_vals'].dropna().str.split(',').to_dict(),'GROUPED_IN', 'Group')\
+ dic_to_ls(redex['country_vals'].dropna().str.split(',').to_dict(),'RESIDES_IN', 'Location')\
+ dic_to_ls(redex['employee_count_quant'].dropna().str.split(',').to_dict(), 'EMPLOYEE_COUNT_BAND', 'Quant_Metric')\
+ dic_to_ls(redex['profit_quant'].dropna().str.split(',').to_dict(), 'PROFIT_BAND', 'Quant_Metric')\
+ dic_to_ls(redex['assets_quant'].dropna().str.split(',').to_dict(), 'ASSETS_BAND', 'Quant_Metric')\
+ dic_to_ls(redex['equity_quant'].dropna().str.split(',').to_dict(), 'EQUITY_BAND', 'Quant_Metric')\
+ dic_to_ls(redex['market_cap_quant'].dropna().str.split(',').to_dict(),'MARKET_CAP_BAND', 'Quant_Metric')
ent_edge_df = pd.DataFrame(ls, columns = ['company_id','val_id','edge_typ', 'node_typ'])
#final etl step specific to my tasks
ent_edge_df = ent_edge_df[~ent_edge_df['val_id'].str.contains('0_')]
ent_edge_df = ent_edge_df[ent_edge_df['val_id']!='']

In [57]:
ent_edge_df

,company_id,val_id,edge_typ,node_typ
5,Q39089708,Q1358919,INDUSTRY,Industry
22,Q98665504,Q11828862,INDUSTRY,Industry
23,Q25936435,Q880371,INDUSTRY,Industry
30,Q466183,Q607081,INDUSTRY,Industry
40,Q50557071,Q8486,INDUSTRY,Industry
...,...,...,...,...
201242,Q219635,1_market_cap,MARKET_CAP_BAND,Quant_Metric
202020,Q467752,2_market_cap,MARKET_CAP_BAND,Quant_Metric
202547,Q156238,3_market_cap,MARKET_CAP_BAND,Quant_Metric
202816,Q192314,2_market_cap,MARKET_CAP_BAND,Quant_Metric


In [10]:
raw_df = pd.read_csv('../data/edges/Q878.csv')
raw_df = clean_wiki(raw_df)
[_ for _ in raw_df.columns if 'value' in _]

['company.value', 'owner.value', 'acquiredate.value']

In [11]:
new_names = {'company.value':'owned_id',
'owner.value':'owner_id',
'acquiredate.value':'date'}

In [12]:
df = pd.DataFrame(columns=list(new_names))
prev_cols = [_ for _ in list(new_names) if _ in raw_df.columns]
df[prev_cols] = raw_df[prev_cols]
df.columns = df.columns.map(new_names)
edge_df = df.copy()
edge_df = edge_df.fillna('')

In [13]:
edge_df

,owned_id,owner_id,date
0,Q61971936,Q55613979,2020-07-01T00:00:00Z
1,Q460373,Q212304,
2,Q2544289,Q4115940,
3,Q4120269,Q543449,


In [16]:
if len(pd.DataFrame()):
    print('hey')